In [1]:
import requests, os, datetime, json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# https://en.wikipedia.org/wiki/1996_Major_League_Soccer_season

# probably don't run again for earlier seasons in case format changes

In [105]:
all_tables = []
for year in range(1996, 2021):
    url = "https://en.wikipedia.org/wiki/{}_Major_League_Soccer_season"

    r = requests.get(url.format(year))
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")
    
    check = False
    headers = []
    for header in soup.find_all(['h1', 'h2', 'h3', 'h4']):
        if 'standings' in header.text.lower():
            check = True
            continue
        if check and ('division' in header.text.lower() or 'conference'in header.text.lower()):
            if 'eastern' in header.text.lower() or 'central' in header.text.lower() or 'western' in header.text.lower():
                headers.append(header.text.split('[')[0])   
        else:
            check = False 
            
    if len(headers) < 2:
        if year == 2001 or year == 2000: # only seasons with 3 conferences
            headers = ['Eastern Conference', 'Central Conference', 'Western Conference']
        else: 
            headers = ['Eastern Conference', 'Western Conference']
        
        # print("couldn't find conferences/divisons for {}, guessing {}".format(year, headers))
    
    headers.append('Overall')
    
    if year == 2011:
        headers = ['Overall', 'Eastern Conference', 'Western Conference']
    
    dfs = pd.read_html(r.text)
    
    i = 0
    table_dfs = []
    for df in dfs:
        if 'Pts' in df.columns:
            df.columns = [x.split('.')[0].replace('vte', '') for x in df.columns]
            df = df.rename(columns={'Club': 'Team', 'Qualification[a]': 'Qualification', 
                     'Unnamed: 10': 'Qualification', 'Pld': 'GP', 'GP*': 'GP', 
                     'T': 'D', 'Western Conference': 'Team', 'Eastern Conference': 'Team', 
                     '(sw)': 'SW', '(sl)': 'SL', 'P': 'GP', 'SOW': 'SW'})
            df['Conference'] = headers[i]
            df['Year'] = year
            table_dfs.append(df)
            i += 1
       
        elif len(df) > 1 and 'Pts' in df.loc[1].values:
            df.columns = df.loc[1].values
            df = df.drop([0, 1]).reset_index(drop=True)
            if np.nan in df.columns:
                df = df.drop(np.nan, axis=1)
            df.columns = [x.split('.')[0].replace('vte', '') for x in df.columns]
            df = df.rename(columns={'Club': 'Team', 'Qualification[a]': 'Qualification', 
                     'Unnamed: 10': 'Qualification', 'Pld': 'GP', 'GP*': 'GP', 
                     'T': 'D', 'Western Conference': 'Team', 'Eastern Conference': 'Team', 
                     '(sw)': 'SW', '(sl)': 'SL', 'P': 'GP', 'SOW': 'SW'})
            df['Conference'] = headers[i]
            df['Year'] = year
            table_dfs.append(df)
            i += 1
        if i == 3 and (year == 2018 or year == 2020): # 2018 has an extra table and 2020 has many
            break
    
    print("year: {}, headers: {}".format(year, headers))
    if len(headers) != len(table_dfs):
        print(len(headers), len(table_dfs))
        
    all_tables.extend(table_dfs)

year: 1996, headers: ['Eastern Conference', 'Western Conference', 'Overall']
year: 1997, headers: ['Eastern Conference', 'Western Conference', 'Overall']
year: 1998, headers: ['Eastern Conference', 'Western Conference', 'Overall']
3 2
year: 1999, headers: ['Eastern Conference', 'Western Conference', 'Overall']
3 2
year: 2000, headers: ['Eastern Division', 'Central Division', 'Western Division', 'Overall']
year: 2001, headers: ['Eastern Conference', 'Central Conference', 'Western Conference', 'Overall']
4 3
year: 2002, headers: ['Eastern Conference', 'Western Conference', 'Overall']
year: 2003, headers: ['Eastern Conference', 'Western Conference', 'Overall']
3 2
year: 2004, headers: ['Eastern Conference', 'Western Conference', 'Overall']
year: 2005, headers: ['Eastern Conference', 'Western Conference', 'Overall']
year: 2006, headers: ['Eastern Conference', 'Western Conference', 'Overall']
year: 2007, headers: ['Eastern Conference', 'Western Conference', 'Overall']
year: 2008, headers: [

In [107]:
big_df = pd.concat(all_tables).reset_index(drop=True)
big_df = big_df.drop(['LIR', 'WIR', 'SO', 'APts'], axis=1)

In [109]:
big_df
# SW - shootout wins
# SL - shootout loses

,Pos,Team,GP,W,L,SW,GF,GA,GD,Pts,Qualification,Conference,Year,SL,D,Head-to-head,PPG
0,1,Tampa Bay Mutiny,32,19,12,1.0,66,51,+15,58,Playoffs,Eastern Conference,1996,NaN,NaN,NaN,NaN
1,2,D.C. United,32,15,16,1.0,62,56,+6,46,Playoffs,Eastern Conference,1996,NaN,NaN,NaN,NaN
2,3,NY/NJ MetroStars,32,12,17,3.0,45,47,−2,39,Playoffs,Eastern Conference,1996,NaN,NaN,NaN,NaN
3,4,Columbus Crew,32,11,17,4.0,59,60,−1,37,Playoffs,Eastern Conference,1996,NaN,NaN,NaN,NaN
4,5,New England Revolution,32,9,17,6.0,43,56,−13,33,NaN,Eastern Conference,1996,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,22,Chicago Fire FC,23,5,10,NaN,33,39,−6,23,NaN,Overall,2020,NaN,8,NaN,1.00
730,23,Atlanta United FC (U),23,6,13,NaN,23,30,−7,22,2021 CONCACAF Champions League[e],Overall,2020,NaN,4,NaN,0.96
731,24,D.C. United,23,5,12,NaN,25,41,−16,21,NaN,Overall,2020,NaN,6,NaN,0.91
732,25,Houston Dynamo,23,4,10,NaN,30,40,−10,21,NaN,Overall,2020,NaN,9,NaN,0.91


In [110]:
big_df.to_csv(os.path.join('data', 'tables', 'all_tables.csv'), index=False)

In [111]:
pd.read_csv(os.path.join('data', 'tables', 'all_tables.csv'))

,Pos,Team,GP,W,L,SW,GF,GA,GD,Pts,Qualification,Conference,Year,SL,D,Head-to-head,PPG
0,1.0,Tampa Bay Mutiny,32,19,12,1.0,66,51,+15,58,Playoffs,Eastern Conference,1996,NaN,NaN,NaN,NaN
1,2.0,D.C. United,32,15,16,1.0,62,56,+6,46,Playoffs,Eastern Conference,1996,NaN,NaN,NaN,NaN
2,3.0,NY/NJ MetroStars,32,12,17,3.0,45,47,−2,39,Playoffs,Eastern Conference,1996,NaN,NaN,NaN,NaN
3,4.0,Columbus Crew,32,11,17,4.0,59,60,−1,37,Playoffs,Eastern Conference,1996,NaN,NaN,NaN,NaN
4,5.0,New England Revolution,32,9,17,6.0,43,56,−13,33,NaN,Eastern Conference,1996,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,22.0,Chicago Fire FC,23,5,10,NaN,33,39,−6,23,NaN,Overall,2020,NaN,8.0,NaN,1.00
730,23.0,Atlanta United FC (U),23,6,13,NaN,23,30,−7,22,2021 CONCACAF Champions League[e],Overall,2020,NaN,4.0,NaN,0.96
731,24.0,D.C. United,23,5,12,NaN,25,41,−16,21,NaN,Overall,2020,NaN,6.0,NaN,0.91
732,25.0,Houston Dynamo,23,4,10,NaN,30,40,−10,21,NaN,Overall,2020,NaN,9.0,NaN,0.91
